# This notebook is for the Fair-Shapely experiment

In [6]:
%reload_ext autoreload
%autoreload 2

### 1. Import libraries

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

### 2. Import dataset

In [9]:
from src.data.unified_dataloader import load_dataset

a, processed_german_credit = load_dataset('german_credit')
# _, processed_uci = load_dataset('uci')

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64


In [10]:
processed_german_credit.head(3)

,Age,sex,Credit amount,Duration,Job_0,Job_1,Job_2,Job_3,Housing_free,Housing_own,...,Checking account_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Risk
0,2.766456,0,-0.745131,-1.236478,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,-1.191404,1,0.949817,2.248194,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,1
2,1.183312,0,-0.416562,-0.738668,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


### 3. Split label/unlabel data, split train/test data.

In [12]:
'''UCI dataset'''
# df = processed_uci.copy()
# X = df.drop('income', axis=1)
# y = df['income']

'''German Credit dataset'''
df = processed_german_credit.copy()
X = df.drop('Risk', axis=1)
y = df['Risk']

# into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25) 
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

X_train shape: (700, 26)
X_test shape: (300, 26)


### 4. Train the original model

In [13]:
model = XGBClassifier()  # 可以替换为 RandomForestClassifier() 等其他模型
model.fit(X_train,y_train)

# 预测和评估
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6766666666666666


In [14]:
X_train['sex'].value_counts()

sex
0    479
1    221
Name: count, dtype: int64

In [28]:
male_condition = X_train['sex'] == 0
X_train_majority = X_train[male_condition]
y_train_majority = y_train[male_condition]

female_condition = X_train['sex'] == 1
X_train_minority = X_train[female_condition]
y_train_minority = y_train[female_condition]

### 5. Evaluate the performance of original model

In [ ]:
from src.attribution.oracle_metric import perturb_numpy_ver
from src.attribution import FairnessExplainer
sen_att_name = ["sex"]
sen_att = [X_train.columns.get_loc(name) for name in sen_att_name]
priv_val = [1]
unpriv_dict = [list(set(X_train.values[:, sa])) for sa in sen_att]
for sa_list, pv in zip(unpriv_dict, priv_val):
    sa_list.remove(pv)
# print(f'sen_att_name:{sen_att_name}')
# print(f'sen_att:{sen_att}') # index of sensitive attribute
# print(f'priv_val:{priv_val}') # privileged value
# print(f'unpriv_dict:{unpriv_dict}') # unprivileged value(all values in the sensitive attribute, except the privileged value)



''' 
计算DR value的函数
'''
def fairness_value_function(sen_att, priv_val, unpriv_dict, X, model):
    X_disturbed = perturb_numpy_ver(
        X=X,
        sen_att=sen_att,
        priv_val=priv_val,
        unpriv_dict=unpriv_dict,
        ratio=1.0,
    )
    fx = model.predict_proba(X)[:, 1]
    fx_q = model.predict_proba(X_disturbed)[:, 1]
    return np.mean(np.abs(fx - fx_q))
original_DR = fairness_value_function(sen_att, priv_val, unpriv_dict, X_test.values, model)
print(f'original_DR: {original_DR}')

original_DR: 0.05330166965723038


### 5. 把female和male匹配，（或者male和female匹配），然后进行修改，重新训练

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from typing import Tuple, List, Dict
from xgboost import XGBClassifier
import pdb
from sklearn.metrics import accuracy_score
from src.matching.ot_matcher import OptimalTransportPolicy
from src.matching.nn_matcher import NearestNeighborDataMatcher
from src.attribution import FairnessExplainer
from src.composition.data_composer import DataComposer
from src.attribution.oracle_metric import perturb_numpy_ver

In [45]:
def fix_negative_probabilities(varphi):
    """
    Fix the probability distribution by:
    1. Setting negative values to 0.
    2. Normalizing the probabilities to sum to 1.
    """
    # Step 1: Set negative values to 0
    varphi = np.maximum(varphi, 0)
    
    # Step 2: Normalize to make the sum equal to 1
    total_prob = varphi.sum()
    if total_prob == 0:
        raise ValueError("All probabilities are zero after fixing negative values.")
    
    varphi = varphi / total_prob
    return varphi

In [31]:
print(f'X_train.shape: {X_train.shape}')
print(f'X_train_majority.shape: {X_train_majority.shape}')
print(f'X_train_minority.shape: {X_train_minority.shape}')

X_train.shape: (700, 26)
X_train_majority.shape: (479, 26)
X_train_minority.shape: (221, 26)


In [25]:
''' Experiment new

1. 从majority parity(此处男性)中随机选择30%, 50%, 70%的比例，作为将被替换的数据集X_train_replace_majority,剩余部分为X_train_rest_majority
2. 将X_train_minority与X_train_replace_majority进行匹配
3. 使用fairshap,把X_train_minority作为baseline dataset，找到X_train_replace_majority中需要替换的数据，假设总共需要替换n个数据点
4. (1,n,20)根据这些,分别计算替换(1,n)中不同个数的结果,把需要替换的数据替换到X_train_replace_majority中,得到X_train_replace_majority_new
5. 把X_train_replace_majority_new和X_train_rest_majority,还有X_train_minority合并,得到新的X_train_new，然后重新训练，得到新的模型model_new，计算新的DR值

----------   循环 30%, 50%, 70%的比例，以及不同的n值，得到DR值的变化  -------------------
'''

' Experiment new\n\n1. 从多数群体(男性)中随机选择30%, 50%, 70%的比例，作为将被替换的数据集X\n1. female(少数)和male(多数)进行匹配\n\n'

In [39]:
# 1. 从majority parity(此处男性)中随机选择30%, 50%, 70%的比例，作为将被替换的数据集X_train_replace_majority,剩余部分为X_train_rest_majority
proportion = 0.2
X_train_replace_majority = X_train_majority.sample(frac=proportion, random_state=20)
X_train_rest_majority = X_train_majority.drop(X_train_replace_majority.index)

y_train_replace_majority = y_train_majority.loc[X_train_replace_majority.index]
y_train_rest_majority = y_train_majority.drop(X_train_replace_majority.index)

# 2. 将X_train_minority与X_train_replace_majority进行匹配
matching = NearestNeighborDataMatcher(X_labeled=X_train_replace_majority, X_unlabeled=X_train_minority).match(n_neighbors=1)
matching.shape

(96, 221)

In [58]:
# 3. 使用fairshap,把X_train_minority作为baseline dataset，找到X_train_replace_majority中需要替换的数据，假设总共需要替换n个数据点
fairness_explainer_original = FairnessExplainer(
    model=model, 
    sen_att=sen_att, 
    priv_val=priv_val, 
    unpriv_dict=unpriv_dict
    )
fairness_shapley_value = fairness_explainer_original.shap_values(
                            X = X_train_replace_majority.values,
                            X_baseline = X_train_minority.values,
                            matching=matching,
                            sample_size=500,
                            shap_sample_size="auto",
                        )
varphi = fix_negative_probabilities(fairness_shapley_value)
non_zero_count =np.count_nonzero(varphi)
print(f'总共可以替换的点数:{non_zero_count}')

q = DataComposer(
                x_counterfactual=X_train_minority.values, 
                joint_prob=matching, 
                method="max").calculate_q()    # q是与X_train_replace_majority匹配的X_train_minority中的数据

总共可以替换的点数:173


In [59]:
# 4. (1,n,20)根据这些,分别计算替换(1,n)中不同个数的结果,把需要替换的数据替换到X_train_replace_majority中,得到X_train_replace_majority_new
gap = 10
values_range = np.arange(1, non_zero_count, 10)
after_values = []
fairness_accuracy_pairs = []
for action_number in values_range:


    # Step 1: 将 varphi 的值和位置展开为一维
    flat_varphi = [(value, row, col) for row, row_vals in enumerate(varphi)
                for col, value in enumerate(row_vals)]

    # Step 2: 按值降序排序
    flat_varphi_sorted = sorted(flat_varphi, key=lambda x: x[0], reverse=True)

    # Step 3: 挑出前 action_number 个数的位置
    top_positions = flat_varphi_sorted[:action_number]

    # Step 4: 替换 X 中前三列的值为 S 中对应位置的值
    for value, row_idx, col_idx in top_positions:
        X_train_replace_majority.iloc[row_idx, col_idx] = q[row_idx, col_idx]
    
    # Step 5: 合并数据集
    X_Train_New = pd.concat([X_train_replace_majority,X_train_rest_majority, X_train_minority], axis=0)
    y_train_new = pd.concat([y_train_replace_majority, y_train_rest_majority, y_train_minority], axis=0)
    # Step 6: Train and evaluate model
    x = X_Train_New
    y = y_train_new
    
    model_new = XGBClassifier()
    model_new.fit(x, y)
    
    # 计算fairness values
    sen_att_name = ['sex']
    sen_att = [X_train.columns.get_loc(name) for name in sen_att_name]
    priv_val = [1]
    unpriv_dict = [list(set(X_train.values[:, sa])) for sa in sen_att]
    for sa_list, pv in zip(unpriv_dict, priv_val):
        sa_list.remove(pv)
    after = fairness_value_function(sen_att, priv_val, unpriv_dict, X_test.values, model_new)
    after_values.append(after)
    original_DR = 0.05330166965723038
    if after < original_DR:
        y_new_pred = model_new.predict(X_test)
        accuracy_new = accuracy_score(y_test, y_new_pred)
        fairness_accuracy_pairs.append((after, accuracy_new, action_number))  # Store both values as a tuple

In [ ]:
plt.figure(figsize=(10, 6))

plt.scatter(values_range, after_values, label='New model', marker='x')
plt.axhline(y=original_DR, color='r', linestyle='--', label='Original DR')
plt.title(f'Proportion: {proportion}', fontsize=10)
plt.xlabel('Limited actions')
plt.ylabel('DR Value')
plt.legend()

plt.show()